**<H1>ЛЕКСИЧНИЙ АНАЛІЗ</H1>**

Автор: **Григорій ЖОЛТКЕВИЧ**\
Автор: **Микола ГОРА**

# **Підготовка блокноту до роботи**

In [9]:
from typing import Tuple, Sequence, Any
from typing_extensions import Self

# **Клас регулярних виразів**

In [10]:
class ReX(tuple):

    EMT = 0  # empty expression
    NIL = 1  # nil expression
    CHR = 2  # one letter expression
    AST = 3  # Kleene closure
    CAT = 4  # concatanation
    ALT = 5  # alternation

    def __new__(cls, kind: int, *args: Sequence[Any]) -> Self:
        if type(kind) != int:
            raise ValueError("ReX() error! Bad type of expression kind")
        if kind == ReX.EMT or kind == ReX.NIL:
            if args:
                raise ValueError("ReX() error! Unexpected argument(s)")
            return super().__new__(cls, (kind, ()))
        elif kind == ReX.CHR:
            if len(args) != 1:
                raise ValueError("ReX() error! Invalid number of arguments")
            if type(args[0]) != str and len(args[0]) != 1:
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        elif kind == ReX.AST:
            if len(args) != 1:
                raise ValueError("ReX() error! Invalid number of arguments")
            if type(args[0]) != ReX:
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        elif kind == ReX.CAT or kind == ReX.ALT:
            if len(args) != 2:
                raise ValueError("ReX() error! Invalid number of arguments")
            if not all(type(arg) == ReX for arg in args):
                raise ValueError("ReX() error! Bad argument(s)")
            return super().__new__(cls, (kind, args))
        else:
            raise ValueError("ReX() error! Bad kind of expression")

    @property
    def kind(self):
        return self[0]

    def __eq__(self, another: Self) -> bool:
        if type(another) != ReX:
            raise ValueError("invalid comparison")
        return super().__eq__(self, another)

    def __ne__(self, another):
        return not (self == another)

    def __str__(self):
        if self.kind == ReX.EMT:
            return "∅"
        if self.kind == ReX.NIL:
            return "ϵ"
        if self.kind == ReX.CHR:
            return self[1][0]
        if self.kind == ReX.AST:
            return f"{self[1][0]}*"
        if self.kind == ReX.CAT:
            return f"({self[1][0]} {self[1][1]})"
        # self.kind == ReX.ALT
        return f"({self[1][0]} | {self[1][1]})"

    @classmethod
    def Empty(cls) -> Self:
        return ReX(ReX.EMT)

    @classmethod
    def Nil(cls) -> Self:
        return ReX(ReX.NIL)

    @classmethod
    def Char(cls, ch: str) -> Self:
        return ReX(ReX.CHR, ch)

    @classmethod
    def Ast(cls, rex: Self) -> Self:
        return ReX(ReX.AST, rex)

    @classmethod
    def Cat(cls, rex1: Self, rex2: Self) -> Self:
        return ReX(ReX.CAT, rex1, rex2)

    @classmethod
    def Alt(cls, rex1: Self, rex2: Self) -> Self:
        return ReX(ReX.ALT, rex1, rex2)

In [11]:
empty = ReX.Empty()
print(f"empty = {empty}")
nil = ReX.Nil()
print(f"nil   = {nil}")
a, b = ReX.Char('a'), ReX.Char('b')
print(f"a  = '{a}', b = '{b}'")
ast = ReX.Ast(a)
print(f"ast   = {ast}")
cat = ReX.Cat(a, b)
print(f"cat   = {cat}")
alt = ReX.Alt(a, b)
print(f"alt   = {alt}")

empty = ∅
nil   = ϵ
a  = 'a', b = 'b'
ast   = a*
cat   = (a b)
alt   = (a | b)


# **Функція** `simplify()`

Функція `simplify(expr: ReX) -> ReX`
забезпечує рекурсивне спрощення регулярних виразів за правилами

- $(\emptyset\mid e)\longrightarrow e$
- $(e\mid\emptyset)\longrightarrow e$
- $(\emptyset\ e)\longrightarrow\emptyset$
- $(e\ \emptyset)\longrightarrow\emptyset$
- $(\epsilon\ e)\longrightarrow e$
- $(e\ \epsilon)\longrightarrow e$
- $(\emptyset)^\ast\longrightarrow\epsilon$
- $(\epsilon)^\ast\longrightarrow\epsilon$

In [12]:
def simplify(rex: ReX) -> ReX:
    if rex.kind == ReX.ALT:
        left = simplify(rex[1][0])
        right = simplify(rex[1][1])
        if left.kind == ReX.EMT:
            return right
        if right.kind == ReX.EMT:
            return left
        return ReX.Alt(left, right)
    elif rex.kind == ReX.CAT:
        left = simplify(rex[1][0])
        right = simplify(rex[1][1])
        if left.kind == ReX.EMT or right.kind == ReX.EMT:
            return ReX.Empty()
        if left.kind == ReX.NIL:
            return right
        if right.kind == ReX.NIL:
            return left
        return ReX.Cat(left, right)
    elif rex.kind == ReX.AST:
        inner = simplify(rex[1][0])
        if inner.kind == ReX.EMT or inner.kind == ReX.NIL:
            return ReX.Nil()
        return ReX.Ast(inner)
    return rex

# **Функція** `check_nil()`

Функція `does_contain_nil(rex: ReX) -> bool`
перевіряє чи належить порожнє слово регулярній мові, що визначається регулярним виразом `ReX`.

In [13]:
def does_contain_nil(rex: ReX) -> bool:
    rex = simplify(rex)
    if rex.kind == ReX.NIL:
        return True
    if rex.kind == ReX.EMT:
        return False
    if rex.kind == ReX.CHR:
        return False
    if rex.kind == ReX.AST:
        return True
    if rex.kind == ReX.CAT:
        return does_contain_nil(rex[1][0]) and does_contain_nil(rex[1][1])
    if rex.kind == ReX.ALT:
        return does_contain_nil(rex[1][0]) or does_contain_nil(rex[1][1])
    raise ValueError("Unknown kind of ReX")

# **Функція** `Brzozowski()`

Функція `Brzozowski(rex: ReX, ch: str) -> ReX`
обчислює похідну Бжозовськи за символом `ch` регулярного виразу `rex`.

In [14]:
def Brzozowski(rex: ReX, ch: str) -> ReX:
    if rex.kind == ReX.EMT:
        return ReX.Empty()
    if rex.kind == ReX.NIL:
        return ReX.Empty()
    if rex.kind == ReX.CHR:
        return ReX.Nil() if rex[1][0] == ch else ReX.Empty()
    if rex.kind == ReX.AST:
        return ReX.Cat(Brzozowski(rex[1][0], ch), rex)
    if rex.kind == ReX.CAT:
        left, right = rex[1]
        if does_contain_nil(left):
            return ReX.Alt(ReX.Cat(Brzozowski(left, ch), right), Brzozowski(right, ch))
        else:
            return ReX.Cat(Brzozowski(left, ch), right)
    if rex.kind == ReX.ALT:
        left, right = rex[1]
        return ReX.Alt(Brzozowski(left, ch), Brzozowski(right, ch))
    raise ValueError("Unknown kind of ReX")

In [15]:
def recognize(rex: ReX, string: str) -> bool:
    for ch in string:
        rex = Brzozowski(rex, ch)
    return does_contain_nil(rex)

# Example usage:
rex = ReX.Cat(ReX.Char('a'), ReX.Ast(ReX.Char('b')))
print(recognize(rex, "abbb"))  # Should return True
print(recognize(rex, "a"))     # Should return True
print(recognize(rex, "ac"))    # Should return False

True
True
False
